In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import load_model

In [ ]:
file_path = 'words.txt'
with open(file_path, 'r') as file:
    words_list = file.read().splitlines()
preprocessed_words = [word.lower() for word in words_list]

In [ ]:
input_output_pairs = []

In [ ]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [ ]:
# Create input-output pairs
input_output_pairs = []
for word in preprocessed_words:
    for letter in alphabet:
        if letter in word:
            input_word = ''.join(['_' if char == letter else char for char in word])
            target_letter = letter
            input_output_pairs.append((input_word, target_letter))

In [ ]:
for word in preprocessed_words:
    for letter1 in alphabet:
        for letter2 in alphabet:
            if letter1 != letter2 and letter1 in word and letter2 in word:
                input_word = ''.join(['_' if (char == letter1 or char == letter2) else char for char in word])
                target_letter1 = letter1
                target_letter2 = letter2
                input_output_pairs.append((input_word, target_letter1))
                input_output_pairs.append((input_word, target_letter2))

In [ ]:
for word in preprocessed_words:
    for letter1 in alphabet:
        for letter2 in alphabet:
            for letter3 in alphabet:
                if letter1 != letter2 and letter1 != letter3 and letter2 != letter3 and letter1 in word and letter2 in word and letter3 in word:
                    input_word = ''.join(['_' if (char == letter1 and ) else char for char in word])
                    target_letter1 = letter1
                    target_letter2 = letter2
                    target_letter3 = letter3
                    input_output_pairs.append((input_word, target_letter1))
                    input_output_pairs.append((input_word, target_letter2))
                    input_output_pairs.append((input_word, target_letter3))

In [ ]:
# # Create input-output pairs
# 
# for word in preprocessed_words:
#     for i in range(len(word)):
#         input_word = word[:i] + '_' + word[i+1:]  # Replace the ith character with '_'
#         target_letter = word[i]  # The missing letter is the ith character
#         input_output_pairs.append((input_word, target_letter))

In [ ]:
# for word in preprocessed_words:
#     for i in range(len(word)-1):
#         input_word = word[:i] + '_' + '_' + word[i+2:]  # Replace the ith character with '_'
#         target_letter = word[i]
#         target_letter2 = word[i+1]# The missing letter is the ith character
#         input_output_pairs.append((input_word, target_letter))
#         input_output_pairs.append((input_word, target_letter2))

In [ ]:
# for word in preprocessed_words:
#     for i in range(len(word)-2):
#         input_word = word[:i] + '_' + '_' + '_' + word[i+3:]  # Replace the ith character with '_'
#         target_letter = word[i]
#         target_letter2 = word[i+1]# The missing letter is the ith character
#         target_letter3 = word[i+2]
#         input_output_pairs.append((input_word, target_letter))
#         input_output_pairs.append((input_word, target_letter2))
#         input_output_pairs.append((input_word, target_letter3))

In [ ]:
# # Create input-output pairs with len >=4
# input_output_pairs = []
# for word in preprocessed_words:
#     if len(word) >= 4:
#         for i in range(len(word)-3):
#             input_word = word[:i] + '_' + '_' + '_' + '_' + word[i+4:]  # Replace the ith character with '_'
#             target_letter = word[i]  # The missing letter is the ith character
#             target_letter2 = word[i+1]
#             target_letter3 = word[i+2]
#             target_letter4 = word[i+3]
#             input_output_pairs.append((input_word, target_letter))
#             input_output_pairs.append((input_word, target_letter2))
#             input_output_pairs.append((input_word, target_letter3))
#             input_output_pairs.append((input_word, target_letter4))

In [ ]:
characters = sorted(set(''.join([pair[0] for pair in input_output_pairs])))
char_to_int = {char: idx for idx, char in enumerate(characters)}
index_to_char = {idx: char for char, idx in char_to_int.items()}

In [ ]:
#vectorizing and padding
input_sequences = []
target_sequences = []
for input_word, target_letter in input_output_pairs:
    input_sequence = [char_to_int[char] for char in input_word]
    target_sequence = [char_to_int[target_letter]]
    input_sequences.append(input_sequence)
    target_sequences.append(target_sequence)

input_sequences = np.array(input_sequences)
target_sequences = np.array(target_sequences)

input_sequences = pad_sequences(input_sequences,maxlen=len(characters), padding='post')
input_sequences = np.reshape(input_sequences, (input_sequences.shape[0], 1, input_sequences.shape[1]))

In [ ]:
#encoder
encoder_inputs = Input(shape=(None, len(characters)))
encoder_lstm = LSTM(128, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

In [ ]:
#decoder
decoder_inputs = Input(shape=(None, len(characters)))
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(27, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit([input_sequences,input_sequences], target_sequences, batch_size=16, epochs=1, verbose=1)

In [ ]:
# Save the model 
model.save('model_LSTM_s2s_10.keras')